Autor: Ricardo Ander-Egg Aguilar

* 🖥: https://ricardoanderegg.com/
* 🐦: https://twitter.com/ricardoanderegg
* 👨🏻‍🎓: https://www.linkedin.com/in/ricardoanderegg/

## ORM, Bases de datos con SQLAlchemy

ORM = Object relational mapper

In [28]:
%%writefile program.py
import mis_funciones

usuarios = mis_funciones.leer_json("users.json")

print("Hola, introduce un nuevo valor")

valor = input("Valor: ")

usuarios["andrea"] = valor

mis_funciones.guardar_json(usuarios, "users_updated.json")

print("JSON actualizado")

Overwriting program.py


In [29]:
!python3 program.py

Hola, introduce un nuevo valor
Valor: 30000
JSON actualizado


In [15]:
usuarios = {"marta": 123, "laura": 345, "andrea": 234}

In [16]:
import json

with open("usuarios.json", "w") as f:
    f.write(json.dumps(usuarios))

In [17]:
%%writefile mis_funciones.py

import json


def leer_json(nombre):
    with open(nombre) as f:
        contenidos_archivo = f.read()
        obj = json.loads(contenidos_archivo)

    return obj


def guardar_json(obj, nombre):
    with open(nombre, "w") as f:
        cadena_json = json.dumps(obj)
        f.write(cadena_json)

Writing mis_funciones.py


In [18]:
from mis_funciones import leer_json, guardar_json

In [19]:
usuarios = leer_json("users.json")

In [21]:
usuarios["andrea"] = 1000

In [22]:
usuarios

{'andrea': 1000, 'laura': 345, 'marta': 123}

In [23]:
guardar_json(usuarios, "users_updated.json")

In [31]:
import sqlite3

In [32]:
con = sqlite3.connect("test.db")

In [33]:
con.execute("create table kv (key string, value string)")

In [51]:
def insert(valor):
    con.execute("insert into kv values ('hola', :valor)", {"valor": valor})
    con.commit()

In [52]:
insert("jajajaj")

In [53]:
con.execute("select * from kv").fetchall()

[('hola', 'adios'),
 ('hola', 'hello'),
 ('hola', 'jajajaj'),
 ('hola', 'jajajaj')]

In [ ]:
## NOOO ##
f"""
INSERT INTO kv VALUES ({}, {})
"""

In [84]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

In [85]:
# creamos una base de datos SQLite directamente en la memoria RAM del ordenador
engine = create_engine("sqlite:///curso.db")

# para crearla en el disco duro. echo=True hará que SQLAlchemy nos devuelve información del tipo de queries que está ejecutando
# engine = create_engine("sqlite:///curso.db", echo=True)

In [86]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

Creamos nuestra tabla.


El método `__repr__` indica dentro de una clase qué queremos que aparezca cuando hagamos `print()`.

In [87]:
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True)
    name = Column(String, index=True)
    fullname = Column(String)
    url = Column(String, unique=True)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', url='%s')>" % (
            self.name,
            self.fullname,
            self.url,
        )

In [88]:
Base.metadata.create_all(engine)

In [89]:
Session = sessionmaker(bind=engine)

In [90]:
session = Session()

In [91]:
ricardo_user = User(name="ricardo", fullname="Ricardo Ander-Egg", url="ricardoanderegg.com")

In [92]:
session.add(ricardo_user)

Ahora podemos hacer queries sin escribir directamente SQL usando python, objetos, y la relación entre estos objetos.

In [93]:
mi_usuario = session.query(User).filter_by(name="ricardo").first()

Añadimos varios a la vez.

In [94]:
lista_users = [
    User(name="jaume", fullname="jaume go", url="twitter.com/jaume"),
    User(name="carles", fullname="carls he", url="twitter.com/carles"),
    User(name="david", fullname="davida b", url="twitter.com/david"),
    User(name="jose antonio", fullname="jose a", url="twitter.com/jose"),
    User(name="pedro", fullname="pedro ro", url="twitter.com/pedro"),
    
]

In [95]:
session.add_all(lista_users)

In [96]:
session.commit()

Si nos equivocamos podemos hacer un **`.rollback()`** antes de hacer commit!

In [97]:
usuario_erroneo = User(name="user erroneo", fullname="oriol mitja", url="google.com")

In [98]:
session.add(usuario_erroneo)

Podemos hacer una query 

In [99]:
session.query(User).filter(User.name.in_(["Manuel", "user erroneo"])).all()

[<User(name='user erroneo', fullname='oriol mitja', url='google.com')>]

Hacemos rollback para "deshacer" la acción que habíamos ejecutado con el `.add()`

In [100]:
session.rollback()

Para filtrar tenemos muchas opciones!

In [101]:
usuarios = session.query(User).filter(User.name.in_(["ricardo", "pedro"])).all()

In [102]:
usuarios[0].url

'twitter.com/pedro'

In [103]:
usuarios[0].url = "twitter.com/ricardoanderegg"

In [104]:
u = usuarios[0]

In [105]:
u

<User(name='pedro', fullname='pedro ro', url='twitter.com/ricardoanderegg')>

In [106]:
session.commit()

In [107]:
for usuario in session.query(User).order_by(User.id):
    print(usuario.name, usuario.fullname, usuario.url)

ricardo Ricardo Ander-Egg ricardoanderegg.com
jaume jaume go twitter.com/jaume
carles carls he twitter.com/carles
david davida b twitter.com/david
jose antonio jose a twitter.com/jose
pedro pedro ro twitter.com/ricardoanderegg


In [108]:
for nombre, nombre_completo in session.query(User.name, User.fullname):
    print(nombre, nombre_completo)

ricardo Ricardo Ander-Egg
jaume jaume go
carles carls he
david davida b
jose antonio jose a
pedro pedro ro


El equivalente de esto en SQL sería:

```sql
SELECT users.name AS nombre,
        users.fullname AS nombre_completo
FROM users
()
```

In [109]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

<User(name='ricardo', fullname='Ricardo Ander-Egg', url='ricardoanderegg.com')> ricardo
<User(name='jaume', fullname='jaume go', url='twitter.com/jaume')> jaume
<User(name='carles', fullname='carls he', url='twitter.com/carles')> carles
<User(name='david', fullname='davida b', url='twitter.com/david')> david
<User(name='jose antonio', fullname='jose a', url='twitter.com/jose')> jose antonio
<User(name='pedro', fullname='pedro ro', url='twitter.com/ricardoanderegg')> pedro


In [110]:
nuevo_user = User(name="jaume", fullname="jaume serra", url="twitter.com/jaume2")

In [111]:
session.add(nuevo_user)

In [112]:
session.commit()

In [114]:
for user in (
    session.query(User)
    .filter(User.name == "jaume")
    .filter(User.fullname == "jaume go")
):
    print(user)

<User(name='jaume', fullname='jaume go', url='twitter.com/jaume')>


In [115]:
nombres_filtro = ["carles", "jaume"]

In [122]:
resultado = session.query(User).filter(User.name.in_(nombres_filtro)).all()

In [123]:
resultado

[<User(name='carles', fullname='carls he', url='twitter.com/carles')>,
 <User(name='jaume', fullname='jaume go', url='twitter.com/jaume')>,
 <User(name='jaume', fullname='jaume serra', url='twitter.com/jaume2')>]

In [ ]:
import requests
import sqlalchemy

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

from sqlalchemy.orm import sessionmaker

In [ ]:
class Lenguajes(Base):
    
    __tablename__ = "lenguajes"
    
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    year = Column(Integer)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
python = Lenguajes(name="python", year=1989)

In [ ]:
session.add(python)

In [ ]:
session.commit()

In [ ]:
session.query(User).all()

[<User(name='ricardo', fullname='Ricardo Ander-Egg', url='twitter.com/ricardoanderegg')>,
 <User(name='jaume', fullname='jaume goñi', url='twitter.com/jaume')>,
 <User(name='carles', fullname='carls hernan', url='twitter.com/carles')>,
 <User(name='david', fullname='davida barnes', url='twitter.com/david')>,
 <User(name='jose antonio', fullname='jose antonio cuenca', url='twitter.com/jose')>,
 <User(name='pedro', fullname='pedro roldar', url='twitter.com/pedro')>,
 <User(name='ricardo', fullname='Ricardo Ander-Egg', url='ricardoanderegg.com')>,
 <User(name='jaume', fullname='jaume goñi', url='twitter.com/jaume')>,
 <User(name='carles', fullname='carls hernan', url='twitter.com/carles')>,
 <User(name='david', fullname='davida barnes', url='twitter.com/david')>,
 <User(name='jose antonio', fullname='jose antonio cuenca', url='twitter.com/jose')>,
 <User(name='pedro', fullname='pedro roldar', url='twitter.com/pedro')>,
 <User(name='jaume', fullname='jaume serra', url='twitter.com/jaume2')

In [ ]:
try:
    session.add(......)
except Exception:
    session.rollback()
    raise

## Ejercicio 01

1. Obtener todas las cartas de la API.
1. La key `colors` contiene un value que es una lista. Filtrar las cartas que contengan `"Green"` en esa lista.
1. Generar una base de datos con SQLAlchemy para almacenar estas cartas.
1. Las columnas que debemos crear son (entre paréntesis está el nombre de la key que tienen en el diccionario):
    * nombre (`name`)
    * ** multiverse_id (`multiverseid`)  <== queda eliminado a menos que queraís hacer la parte extra
    * tipo (`type`)
    * **  url_imagen (`imageUrl`) <== queda eliminado a menos que queraís hacer la parte extra
    * rareza (`rarity`)
    * **Extra**: en las cartas verdes que NO tienen `multiverseid`, crearlo y darle el valor `0`
    
1. De la lista de cartas filtradas que hemos obtenido en el punto $2$, guardarlas todas en la base de datos.

**Extra 🔥**

* Crear una función que obtenga la información de la carta de la base de datos en base a un `id` o como queráis.
* En las cartas que no tengan la variable `"imageUrl"`, crearla y poner: http://placegoat.com/200/200
* Descargar la imagen de esta carta (o de la cabra 😂) y guardarla en un archivo en el disco.
* Crear un archivo JSON que sea una lista de diccionarios con el siguiente formato:

```python
[
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    .
    .
    .
]
```

Por ejemplo:

```python
req_img = requests.get(v[0]["imageUrl"])

with open("imagen.jpg", "wb") as f:
    f.write(req_img.content)
```

In [ ]:
import json

with open("cartas.json", "w") as f:
    f.write(json.dumps(cartas))

In [ ]:
### CODE